# 循环网络模型

**Word2Vec** 词向量是**自然语言处理**（Natural Language Process, NLP）领域的一个重要发明，实现了上下文词义的关联。

但是，它在通往真正的语言理解道路上存在两个瓶颈：

* **无法理解语序**：对于 Word2Vec 来说，“老鼠爱大米”和“大米爱老鼠”生成的上下文特征几乎是一样的。由于它在训练时将上下文组成词袋，词语出现的先后顺序这一重要信息被丢失了。

* **缺乏长程记忆**：Word2Vec 的视野被局限在了一个狭小的“上下文窗口”内。超出了这个范围的单词，模型就完全无法建立起词义间的关联。比如这句歌词：“我爱你就像老鼠爱大米”。Word2Vec 在词袋大小为 5 的情况下，就无法学习到“我爱你”和“老鼠爱大米”之间的关联。

---

怎样让机器拥有“长期记忆”，并不是一个新问题。

1986 年：语言学家**埃尔曼**（Elman）提出了**简单循环网络结构**（Simple RNN）。埃尔曼提出将**隐状态**（Hidden State）反馈到下一个时刻，这成为了现代 **循环神经网络**（Recurrent Neural Network, RNN）的标准范式：模型不仅看当前输入，也参考过去的自己。

所谓**隐状态**，就是在网络模型中，输入数据经过层层变换，准备进入最后的输出层时的状态。隐状态保存了网络模型到此刻为止，学习到的全部知识，就是所谓的**时间记忆**（temporal memory）。

回头看我们的词向量网络模型，只有两层：嵌入层和阿输出层。其中嵌入层的输出就是隐状态。

简单地讲，**循环神经网络**就是将上一次迭代进入输出层前的隐状态，连接到本次迭代输入层（嵌入层）后的词向量中。这样，一次迭代将包括四个步骤：

* 嵌入层对输入数据进行初步学习，生成词向量；
* 连接词向量和上次迭代的隐状态；
* 对叠加了时间记忆的词向量进一步学习，生成本次迭代的隐状态；
* 输出层生成预测值。

最终的输出数据包括**预测值**和**隐状态**两个部分。预测值用于损失函数开始梯度计算和反向传播；隐状态则参与到下一次迭代。

In [1]:
import csv
import re
from abc import abstractmethod, ABC

import numpy as np

np.random.seed(99)

## 基础架构

### 张量

从现在开始，我们将逐步扩展张量间的直接数学运算。引入的第一个张量运算是：**连接**（concat）。

本质上，我们可以创建一个新的层，来完成两个张量的连接。直接扩展张量来完成运算是一种更高效的做法。这类扩展的张量运算需要符合层的**前向传播函数**（forward）的规范：

* 计算**输出值**（p）；
* 定义**梯度函数**（gradient_fn）；
* 定义**父节点列表**（parents）。

同时，这类扩展的张量运算不能有自己的权重和偏置，因此不需要定义**参数列表**（parameters）。

张量**连接**（concat）的作用是将两个张量的数据（数组）连接成为一个新的数组，我们直接调用 NumPy 的 **连接函数**（concatenate）来完成运算。

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    @property
    def size(self):
        return np.prod(self.data.shape[1:])

    def __repr__(self):
        return f'Tensor({self.data})'

    def concat(self, other, axis):
        p = Tensor(np.concatenate([self.data, other.data], axis=axis))

        def gradient_fn():
            grad = np.split(p.grad, [self.data.shape[axis]], axis=axis)
            self.grad += grad[0]
            other.grad += grad[1]

        p.gradient_fn = gradient_fn
        p.parents = {self, other}
        return p

### 基础数据集

In [3]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size

        self.test_labels = self.test_features = None
        self.train_labels = self.train_features = None

        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size, Tensor(self.labels).size

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size

        feature = Tensor(self.features[start: end])
        label = Tensor(self.labels[start: end])
        return feature, label

    def estimate(self, predictions):
        pass

### 基础层

**循环神经网络**的输出层将输出预测值和隐状态两部分，而我们需要在某一层将隐状态作为输入的一部分。因此我们对基础层的**前向传播函数**（forward）做了一些调整，允许多个输入值作为参数。

In [4]:
class Layer(ABC):

    def __init__(self):
        self.training = True

    def __call__(self, *args):
        return self.forward(*args)

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    @abstractmethod
    def forward(self, *args):
        pass

    @property
    def parameters(self):
        return []

    def __repr__(self):
        return ''

### 基础损失函数

In [5]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### 基础优化器

In [6]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def reset(self):
        for p in self.parameters:
            p.grad = np.zeros_like(p.data)

    @abstractmethod
    def step(self):
        pass

### 基础模型

In [7]:
class Model(ABC):

    def __init__(self, layer, loss_fn, optimizer):
        self.layer = layer
        self.loss_fn = loss_fn
        self.optimizer = optimizer

    @abstractmethod
    def train(self, dataset, epochs):
        pass

    @abstractmethod
    def test(self, dataset):
        pass

## 数据

### IMDB 数据集

In [8]:
class IMDBDataset(Dataset):

    def __init__(self, filename, sequence_length=5):
        self.filename = filename
        self.sequence_length = sequence_length
        super().__init__()

    def load(self):
        self.reviews = []
        self.sentiments = []
        with open(self.filename, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)
            for _, row in enumerate(reader):
                self.reviews.append(row[0])
                self.sentiments.append(row[1])

        split_reviews = []
        for line in self.reviews:
            split_reviews.append(self.clean_text(line.lower()).split())

        self.vocabulary = set(word for line in split_reviews for word in line)
        self.word2index = {word: index for index, word in enumerate(self.vocabulary)}
        self.index2word = {index: word for index, word in enumerate(self.vocabulary)}
        self.tokens = [[self.word2index[word] for word in line if word in self.word2index] for line in split_reviews]

        self.train_features, self.train_labels = self.create_sequence(self.tokens[:-20])
        self.test_features, self.test_labels = self.create_sequence(self.tokens[-20:])

    @staticmethod
    def clean_text(text):
        text = re.sub(r'<[^>]+>', '', text)
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        return text

    def create_sequence(self, tokens):
        predict_position = self.sequence_length // 2
        features, labels = [], []
        for line in tokens:
            for index in range(len(line) - self.sequence_length + 1):
                feature = []
                for i in range(self.sequence_length):
                    if i != predict_position:
                        feature.append(line[index + i])
                features.append(feature)
                labels.append(self.onehot(line[index + predict_position]))
        return features, labels


    def encode(self, text):
        words = self.clean_text(text.lower()).split()
        return [self.word2index[word] for word in words]

    def decode(self, tokens):
        return " ".join([self.index2word[index] for index in tokens])

    def onehot(self, token):
        ebd = np.zeros(len(self.vocabulary))
        ebd[token] = 1
        return ebd

    @staticmethod
    def argmax(vector):
        return np.argmax(vector)

    def estimate(self, predictions):
        count = 0
        for i in range(len(predictions)):
            if self.argmax(predictions[i].data[0]) == self.argmax(self.labels[i].data):
                count += 1
        return count / len(predictions)

## 模型

### 线性层

In [9]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        super().__init__()
        self.weight = Tensor(np.random.randn(out_size, in_size) * np.sqrt(2 / in_size))
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad.T @ x.data
            self.bias.grad += np.sum(p.grad, axis=0)
            x.grad += p.grad @ self.weight.data

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    @property
    def parameters(self):
        return [self.weight, self.bias]

    def __repr__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 顺序层

In [10]:
class Sequential(Layer):

    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def train(self):
        for l in self.layers:
            l.train()

    def eval(self):
        for l in self.layers:
            l.eval()

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    @property
    def parameters(self):
        return [p for l in self.layers for p in l.parameters]

    def __repr__(self):
        return '\n'.join(str(l) for l in self.layers if str(l))

### 嵌入层

In [11]:
class Embedding(Layer):

    def __init__(self, vocabulary_size, embedding_size):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size

        self.weight = Tensor(np.random.randn(embedding_size, vocabulary_size) * np.sqrt(2 / vocabulary_size))

    def forward(self, x: Tensor):
        p = Tensor(np.mean(self.weight.data.T[x.data], axis=1))

        def gradient_fn():
            if type(self.weight.grad) is not np.ndarray:
                self.weight.grad = np.zeros_like(self.weight.data)
            self.weight.grad.T[x.data] += p.grad / len(x.data)

        p.gradient_fn = gradient_fn
        p.parents = {self.weight}
        return p

    @property
    def parameters(self):
        return [self.weight]

    def __repr__(self):
        return f'Embedding[weight{self.weight.data.shape}; vocabulary={self.vocabulary_size}; embedding={self.embedding_size}]'

### ReLU 激活函数

In [12]:
class ReLU(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.maximum(0, x.data))

        def gradient_fn():
            x.grad += p.grad * (p.data > 0)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'ReLU[]'

### Tanh 激活函数

In [13]:
class Tanh(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.tanh(x.data))

        def gradient_fn():
            x.grad += p.grad * (1 - p.data ** 2)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'Tanh[]'

### Sigmoid 激活函数

In [14]:
class Sigmoid(Layer):

    def __init__(self, clip_range=(-100, 100)):
        super().__init__()
        self.clip_range = clip_range

    def forward(self, x: Tensor):
        z = np.clip(x.data, self.clip_range[0], self.clip_range[1])
        p = Tensor(1 / (1 + np.exp(-z)))

        def gradient_fn():
            x.grad += p.grad * p.data * (1 - p.data)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'Sigmoid[]'

### Softmax 激活函数

In [15]:
class Softmax(Layer):

    def __init__(self, axis=-1):
        super().__init__()
        self.axis = axis

    def forward(self, x: Tensor):
        exp = np.exp(x.data - np.max(x.data, axis=self.axis, keepdims=True))
        p = Tensor(exp / np.sum(exp, axis=self.axis, keepdims=True))

        def gradient_fn():
            grad = np.sum(p.data * p.grad, axis=self.axis, keepdims=True)
            x.grad += p.data * (p.grad - grad)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __repr__(self):
        return f'Softmax[]'

### 循环层

为了方便处理隐状态，我们创建了新的复合层，**循环层**（Recurrent Layer）。

循环层包括三个子层（和附加的计划函数层）：嵌入层、隐藏层和输出层。

我们使用向量**连接函数**（concat）将前一次迭代的隐状态，和嵌入层输出的词向量连接成一个新的张量（两倍大小），作为隐藏层的输入。而隐藏层的作用是将这个新的向量再次压缩到词向量的大小，成为本次迭代的隐状态。

因此，循环层的**前向传播函数**（forward）需要输入值和隐状态两个参数。

In [16]:
class Recurrent(Layer):

    def __init__(self, vocabulary_size, embedding_size):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size

        self.embedding = Embedding(vocabulary_size, embedding_size)
        self.hidden = Linear(embedding_size * 2, embedding_size)
        self.output = Linear(embedding_size, vocabulary_size)
        self.tanh = Tanh()

        self.layers = [self.embedding,
                       self.hidden,
                       self.output,
                       self.tanh]

    def forward(self, x: Tensor, h: Tensor):
        if h is None:
            h = Tensor(np.zeros((x.data.shape[0], self.embedding_size)))

        embedding_feature = self.embedding(x)
        concat_feature = embedding_feature.concat(h, axis=1)
        hidden_feature = self.tanh(self.hidden(concat_feature))

        return self.output(hidden_feature), hidden_feature

    def parameters(self):
        return [p for l in self.layers for p in l.parameters]

    def __str__(self):
        return '\n'.join(str(l) for l in self.layers if str(l))

### 损失函数（交叉熵）

In [17]:
class CELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        exp = np.exp(p.data - np.max(p.data, axis=-1, keepdims=True))
        softmax = exp / np.sum(exp, axis=-1, keepdims=True)

        log = np.log(np.clip(softmax, 1e-10, 1))
        ce = Tensor(0 - np.sum(y.data * log) / len(y.data))

        def gradient_fn():
            p.grad += (softmax - y.data) / len(y.data)

        ce.gradient_fn = gradient_fn
        ce.parents = {p}
        return ce

### 损失函数（二元交叉熵）

In [18]:
class BCELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        clipped = np.clip(p.data, 1e-7, 1 - 1e-7)
        bce = Tensor(-np.mean(y.data * np.log(clipped) + (1 - y.data) * np.log(1 - clipped)))

        def gradient_fn():
            p.grad += (clipped - y.data) / (clipped * (1 - clipped)) / len(y.data)

        bce.gradient_fn = gradient_fn
        bce.parents = {p}
        return bce

### 优化器（随机梯度下降）

In [19]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

### 神经网络模型

对于**循环神经网络**，我们需要一个新的模型。主要的区别就是层使用两个参数，其中一个（隐状态）从上次迭代的输出加入到本次迭代的输入。这也是**循环神经网络**的名称来源。

随着使用隐状态将本次迭代和上次迭代连接起来，本次迭代的动态计算图就通过隐状态，连通了上次迭代的动态计算图，并进而依次连通了更早迭代的动态计算图，直至连通第一次迭代的动态计算图。

也就是说，当我们通过损失函数开启梯度计算链路时，梯度将沿着动态计算图依次更新到上次迭代，以及更早的迭代。这种现象称为**随时间反向传播**（Backpropagation Through Time, BPTT）。

随时间反向传播是 RNN 从**时间记忆**中学习的具体表现。但是也存在一个致命的问题，就是梯度计算链路会迅速变长，造成模型训练变得缓慢无比。

实践中的折中方案就是**截断式 BPTT**（Truncated BPTT），也就是在向前连通若干次迭代以后，主动截断继续向前的连通。这个方案解决了训练缓慢的问题，同时也将**时间记忆**从某个时间点截断了。

我们也采用这种截断式 BPTT 的方案，只向前再连通一个词袋的距离。

In [20]:
class RNNModel(Model):

    def train(self, dataset, epochs):
        self.layer.train()
        dataset.train()

        for epoch in range(epochs):
            hidden = None
            for i in range(len(dataset)):
                feature, label = dataset[i]

                # 隐状态
                prediction, hidden = self.layer(feature, hidden)
                loss = self.loss_fn(prediction, label)
                self.optimizer.reset()
                loss.backward()
                self.optimizer.step()
                # 截断动态计算图
                hidden = Tensor(hidden.data)


    def test(self, dataset):
        self.layer.eval()
        dataset.eval()

        predictions = []
        hidden = None
        for i in range(len(dataset)):
            feature, label = dataset[i]
            prediction, hidden = self.layer(feature, hidden)
            predictions.append(prediction)
        return predictions

## 设置

### 学习率

In [21]:
LEARNING_RATE = 0.01

### 轮次

In [22]:
EPOCHS = 100

### 序列长度

In [23]:
SEQUENCE_LENGTH = 5

## 训练

### 迭代

我们已经将**循环神经网络**包装成循环层，因此我们的模型值只需要单独一个循环层就可以了。

**循环神经网络**仍然是多元分类问题，我们继续使用**交叉熵损失函数**（CELoss）。

In [24]:
dataset = IMDBDataset('tinyimdb.csv', sequence_length=SEQUENCE_LENGTH)
layer = Recurrent(len(dataset.vocabulary), 32)
loss = CELoss()
optimizer = SGDOptimizer(layer.parameters(), lr=LEARNING_RATE)

model = RNNModel(layer, loss, optimizer)
model.train(dataset, EPOCHS)
print(layer)

Embedding[weight(32, 86); vocabulary=86; embedding=32]
Linear[weight(32, 64); bias(32,)]
Linear[weight(86, 32); bias(86,)]
Tanh[]


## 验证

### 测试

In [25]:
predictions = model.test(dataset)
print(f'Accuracy: {dataset.estimate(predictions)}')

Accuracy: 0.6025641025641025


测试结果证明，即使在一个非常小的数据集上，**循环神经网络**也已经有明显更好的表现。

### 对比

In [26]:
features, labels = dataset.items()
for i in range(len(predictions)):
    pos = np.argmax(predictions[i].data[0])
    print(f'Feature: {dataset.decode(features.data[i])} | '
          f'Label: {dataset.decode([dataset.argmax(labels.data[i])])} | '
          f'Prediction: {dataset.decode([dataset.argmax(predictions[i].data[0])])}')

Feature: this movie excellent i | Label: was | Prediction: was
Feature: movie was i loved | Label: excellent | Prediction: fantastic
Feature: was excellent loved the | Label: i | Prediction: i
Feature: excellent i the story | Label: loved | Prediction: loved
Feature: i loved story and | Label: the | Prediction: the
Feature: loved the and effect | Label: story | Prediction: story
Feature: the story effect the | Label: and | Prediction: and
Feature: story and the actress | Label: effect | Prediction: character
Feature: and effect actress was | Label: the | Prediction: the
Feature: effect the was wonderful | Label: actress | Prediction: screenplay
Feature: the actress wonderful recommend | Label: was | Prediction: was
Feature: actress was recommend actor | Label: wonderful | Prediction: fantastic
Feature: was wonderful actor character | Label: recommend | Prediction: screenplay
Feature: wonderful recommend character scene | Label: actor | Prediction: actor
Feature: recommend actor scene p

## 课后练习

试一试调整超参数，特别是序列长度，看看对于模型预测准确率有什么影响？